In [1]:
import numpy as np
import sys
import itertools as itertools
sys.path.append("..")
sys.path.append("../..")

import matplotlib.pyplot as plt
from python.Heisenberg import XXZ_model, nn_Heisenberg_model, XY_model
from python.DMRG import DMRG
from python.Zippers import MPO_to_Hamiltonian, contract_MPS
from python.Canonical_Form import get_Neumann_entropy

In [2]:
NKeep = 20
NSweep = 10
Krylov_bases = 5  #* 5 is usually enough
Lanczos_cutoff = 1e-2
iterative_diag = True
two_site = True
verbose = True

# XXZ model

In [2]:
n_sites = 10

ZZ_coupling = 1.0
XY_coupling = 1.0
magnetic_field = 0.0

Hamiltonian = XXZ_model(
    n_sites=n_sites, ZZ_coupling=ZZ_coupling,
    XY_coupling=XY_coupling, magnetic_field=magnetic_field,
)
MPO_transposed = [ham.transpose(2, 3, 0, 1) for ham in Hamiltonian]

In [ ]:
"""
Get exact matrix (exponential barrier)
"""

matrix = MPO_to_Hamiltonian(MPO_transposed)
eigvals, _ = np.linalg.eigh(matrix)
eigvals[:10]

In [ ]:
ground_energies, ground_times, ground_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = iterative_diag,
    two_site = two_site,
    verbose = verbose,
)

eigvals[0]

L=10 | NKeep=20 | NSweep=10 | iterative=True | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-7.1778373 | time=0.0s
iter=1 | energy=-7.1805011 | time=0.296s
iter=2 | energy=-7.1805011 | time=0.409s
iter=3 | energy=-7.1805011 | time=0.567s
iter=4 | energy=-7.1805011 | time=0.739s
iter=5 | energy=-7.1805011 | time=0.901s
iter=6 | energy=-7.1805011 | time=1.02s
iter=7 | energy=-7.1805011 | time=1.23s
iter=8 | energy=-7.1805011 | time=1.38s
iter=9 | energy=-7.1805011 | time=1.52s
iter=10 | energy=-7.1805011 | time=1.62s


In [ ]:
excited_energies, excited_times, excited_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = False,
    orthogonal_to_list_of_MPS=[ground_MPS],
    two_site = two_site,
    verbose = verbose,
)

eigvals[1]

L=10 | NKeep=20 | NSweep=10 | iterative=False | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-0.064555886 | time=0.0s
iter=1 | energy=-6.7185007 | time=1.24s
iter=2 | energy=-6.7185007 | time=2.03s
iter=3 | energy=-6.7185007 | time=3.05s
iter=4 | energy=-6.7185007 | time=3.52s
iter=5 | energy=-6.7185007 | time=4.12s
iter=6 | energy=-6.7185007 | time=4.56s
iter=7 | energy=-6.7185007 | time=5.3s
iter=8 | energy=-6.7185007 | time=6.15s
iter=9 | energy=-6.7185007 | time=7.31s
iter=10 | energy=-6.7185007 | time=8.51s


# nn Heisenberg

In [8]:
n_sites = 10

J1 = 1.0
J2 = 1.0

Hamiltonian = nn_Heisenberg_model(
    n_sites=n_sites, J1=J1, J2=J2,
)
MPO_transposed = [ham.transpose(2, 3, 0, 1) for ham in Hamiltonian]

In [ ]:
"""
Get exact matrix (exponential barrier)
"""

matrix = MPO_to_Hamiltonian(MPO_transposed)
eigvals, _ = np.linalg.eigh(matrix)
eigvals[:10]

array([-2.25, -2.25, -1.75, -1.75, -1.75, -1.75, -1.75, -1.75, -1.75,
       -1.75])

In [13]:
NKeep = 20
NSweep = 10
Krylov_bases = 5 #* 5 is usually enough
Lanczos_cutoff = 1e-2
iterative_diag = True
two_site = True
verbose = True

In [14]:
ground_energies, ground_times, ground_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = iterative_diag,
    two_site = two_site,
    verbose = verbose,
)

eigvals[0]

L=10 | NKeep=20 | NSweep=10 | iterative=True | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-2.25 | time=0.0s
iter=1 | energy=-2.25 | time=0.0263s
iter=2 | energy=-2.25 | time=0.0519s
iter=3 | energy=-2.25 | time=0.0776s
iter=4 | energy=-2.25 | time=0.0979s
iter=5 | energy=-2.25 | time=0.117s
iter=6 | energy=-2.25 | time=0.136s
iter=7 | energy=-2.25 | time=0.157s
iter=8 | energy=-2.25 | time=0.176s
iter=9 | energy=-2.25 | time=0.196s
iter=10 | energy=-2.25 | time=0.215s


-2.25

In [15]:
excited_energies, excited_times, excited_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = False,
    orthogonal_to_list_of_MPS=[ground_MPS],
    two_site = two_site,
    verbose = verbose,
)

eigvals[1]

L=10 | NKeep=20 | NSweep=10 | iterative=False | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=0.050230742 | time=0.0s
iter=1 | energy=-2.2499805 | time=0.208s
iter=2 | energy=-2.2499805 | time=0.352s
iter=3 | energy=-2.2499805 | time=0.462s
iter=4 | energy=-2.2499805 | time=0.674s
iter=5 | energy=-2.2499805 | time=0.799s
iter=6 | energy=-2.2499805 | time=0.871s
iter=7 | energy=-2.2499805 | time=0.997s
iter=8 | energy=-2.2499805 | time=1.06s
iter=9 | energy=-2.2499805 | time=1.13s
iter=10 | energy=-2.2499805 | time=1.19s


-2.25

# XY model

In [6]:
n_sites = 10

J = -1.0
Gamma = 1.0

Hamiltonian = XY_model(
    n_sites=n_sites, J=J, Gamma=Gamma,
)
MPO_transposed = [ham.transpose(2, 3, 0, 1) for ham in Hamiltonian]

In [7]:
"""
Get exact matrix (exponential barrier)
"""

matrix = MPO_to_Hamiltonian(MPO_transposed)
eigvals, _ = np.linalg.eigh(matrix)
eigvals[:10]

array([-5.28482978, -4.75150298, -4.66134396, -4.53573529, -4.39498427,
       -4.25373069, -4.12801717, -4.12188325, -4.00629712, -4.00240849])

In [8]:
ground_energies, ground_times, ground_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = iterative_diag,
    two_site = two_site,
    verbose = verbose,
)

eigvals[0]

L=10 | NKeep=20 | NSweep=10 | iterative=True | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-5.2770168 | time=0.0s
iter=1 | energy=-5.2848298 | time=0.0587s
iter=2 | energy=-5.2848298 | time=0.0893s
iter=3 | energy=-5.2848298 | time=0.118s
iter=4 | energy=-5.2848298 | time=0.148s
iter=5 | energy=-5.2848298 | time=0.178s
iter=6 | energy=-5.2848298 | time=0.208s
iter=7 | energy=-5.2848298 | time=0.236s
iter=8 | energy=-5.2848298 | time=0.265s
iter=9 | energy=-5.2848298 | time=0.304s
iter=10 | energy=-5.2848298 | time=0.345s


np.float64(-5.2848297789078025)

In [9]:
excited_energies, excited_times, excited_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = False,
    orthogonal_to_list_of_MPS=[ground_MPS],
    two_site = two_site,
    verbose = verbose,
)

eigvals[1]

L=10 | NKeep=20 | NSweep=10 | iterative=False | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-0.043132269 | time=0.0s
iter=1 | energy=-4.7511231 | time=0.161s
iter=2 | energy=-4.7514289 | time=0.269s
iter=3 | energy=-4.7514289 | time=0.366s
iter=4 | energy=-4.7514289 | time=0.468s
iter=5 | energy=-4.7514289 | time=0.581s
iter=6 | energy=-4.7514289 | time=0.69s
iter=7 | energy=-4.7514289 | time=0.777s
iter=8 | energy=-4.7514289 | time=0.871s
iter=9 | energy=-4.7514289 | time=0.955s
iter=10 | energy=-4.7514289 | time=1.05s


np.float64(-4.751502980728416)

In [10]:
get_Neumann_entropy(ground_MPS)

array([0.11924839, 0.12692302, 0.1279663 , 0.1281325 , 0.12815813,
       0.12813472, 0.12797017, 0.12694886, 0.11927737])

In [11]:
get_Neumann_entropy(excited_MPS)

array([0.2442514 , 0.49815053, 0.77345297, 0.97314738, 1.0329971 ,
       0.93625976, 0.71811735, 0.45222302, 0.2233231 ])

In [16]:
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

L = 40
N_keep = 32
sweeps = 10
krylov = 5
gammas = np.linspace(0, 1, 21)
Js     = np.linspace(0.2 ,2, 19)

In [17]:
E0  = np.zeros((len(Js), len(gammas)))
gap = np.zeros_like(E0)
SvN = np.zeros_like(E0)

for iJ, J in enumerate(tqdm(Js, desc='J loop')):
    for ig, gamma in enumerate(tqdm(gammas, desc='gamma loop', leave=False)):
        # --- build MPO ------------
        H_MPO = XY_model(n_sites=L, J=J, Gamma=gamma)

        # --- ground state ---------------
        (E_series, _, GS_MPS) = DMRG(
            Hamiltonian = H_MPO,
            NKeep       = N_keep,
            NSweep      = sweeps,
            Krylov_bases= krylov,
            iterative_diag = True,
            two_site    = True,
            verbose     = False,
        )
        E0[iJ, ig] = E_series[-1].real

        # --- first excited state (orthogonalized one further run) ----------
        (E1_series, _, _) = DMRG(
            Hamiltonian = H_MPO,
            NKeep       = N_keep,
            NSweep      = sweeps,
            Krylov_bases= krylov,
            iterative_diag = False,
            two_site    = True,
            orthogonal_to_list_of_MPS = [GS_MPS],  # ⟂ to ground state
            verbose     = False,
        )
        gap[iJ, ig] = (E1_series[-1] - E0[iJ, ig]).real

        # --- mid-chain entanglement entropy -----------
        SvN_chain = get_Neumann_entropy(GS_MPS)
        SvN[iJ, ig] = SvN_chain[L//2 - 1]           # cut at L/2

# --------------------- plotting --------------------------------
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

titles   = [r'Ground-state energy density $e_0$',
            r'Spectral gap $\Delta$',
            r'Mid-chain entropy $S_{\mathrm{vN}}$']
data     = [E0 / L, gap, SvN]

for ax, dat, title in zip(axes, data, titles):
    im = ax.imshow(dat,
                   origin='lower',
                   extent=[gammas[0], gammas[-1],
                               Js[0],    Js[-1]],
                   aspect='auto')
    ax.set_title(title, fontsize=13)
    ax.set_xlabel(r'$\gamma$')
    ax.set_ylabel(r'$J$')
    fig.colorbar(im, ax=ax, fraction=0.046)

plt.tight_layout()
plt.show()

J loop:   0%|          | 0/19 [00:00<?, ?it/s]

gamma loop:   0%|          | 0/21 [00:00<?, ?it/s]

gamma loop:   0%|          | 0/21 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import pickle, datetime, pathlib

results = {
    "timestamp"   : datetime.datetime.now().isoformat(),
    "system_size" : L,
    "bond_dim"    : N_keep,
    "sweeps"      : sweeps,
    "gamma_vals"  : gammas,
    "J_vals"      : Js,
    "E0"          : E0,            # ground-state energy
    "gap"         : gap,           # spectral gap
    "SvN"         : SvN,           # mid-chain von-Neumann entropy
}

fname = pathlib.Path(f"XY_phase_diag_L{L}_chi{N_keep}.pkl").as_posix()
with open(fname, "wb") as f:
    pickle.dump(results, f)

print(f"✔  Saved to {fname}")